In [3]:
import pandas as pd
import numpy as np
from category_encoders.one_hot import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [4]:
from sklearn.tree import DecisionTreeClassifier

In [5]:
df = pd.read_csv('train.csv')

In [102]:
df = df.iloc[:50000]

In [6]:
df_copy = df.copy()

In [7]:
df_copy.dropna(subset=['cap-surface', 'gill-attachment', 'gill-spacing'],inplace = True)

In [8]:
df_copy.isnull().sum()

id                            0
class                         0
cap-diameter                  0
cap-shape                     6
cap-surface                   0
cap-color                     3
does-bruise-or-bleed          2
gill-attachment               0
gill-spacing                  0
gill-color                    7
stem-height                   0
stem-width                    0
stem-root               1135286
stem-surface             659500
stem-color                   13
veil-type               1201556
veil-color              1149287
has-ring                     10
ring-type                 48386
spore-print-color       1141264
habitat                      11
season                        0
dtype: int64

In [430]:
df_copy.dropna(subset=['cap-shape', 'cap-color', 'does-bruise-or-bleed','gill-color'],inplace = True)

In [431]:
y = df_copy["class"]
y = y.map({"p":-1, "e":1})

In [432]:
X = df_copy.drop(["class",'id'], axis=1)

In [44]:
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor, plot_tree

In [433]:
encoder_2 = OneHotEncoder(return_df=True)
X2 = encoder_2.fit_transform(X,y)

In [434]:
Xtrain_1, Xtest_1, ytrain_1, ytest_1 = train_test_split(X2, y, test_size=0.3, random_state=42)

In [435]:
dt =  DecisionTreeClassifier(criterion='entropy', max_depth=15, random_state=18)

In [436]:
dt.fit(Xtrain_1, ytrain_1)

y_train_dt = dt.predict(Xtrain_1)
y_test_dt = dt.predict(Xtest_1)

In [437]:
print(f"Decision tree accuracy train: {accuracy_score(y_train_dt, ytrain_1):.8f}")
print(f"Decision tree accuracy test: {accuracy_score(y_test_dt, ytest_1):.8f}")

Decision tree accuracy train: 0.99080606
Decision tree accuracy test: 0.98873800


In [394]:
df_1 = pd.read_csv('train.csv')


In [395]:
df_1 = df_1.iloc[50000:100000]

In [362]:
df_1 = df_1.iloc[50000:72229]

In [396]:
df_copy_1 = df_1.copy()

In [397]:
df_copy_1.isnull().sum()

id                          0
class                       0
cap-diameter                0
cap-shape                   0
cap-surface             10962
cap-color                   0
does-bruise-or-bleed        0
gill-attachment          8331
gill-spacing            20207
gill-color                  0
stem-height                 0
stem-width                  0
stem-root               44228
stem-surface            31805
stem-color                  0
veil-type               47409
veil-color              43971
has-ring                    0
ring-type                2038
spore-print-color       45804
habitat                     0
season                      0
dtype: int64

In [398]:
df_copy_1.dropna(subset=['cap-surface', 'gill-attachment', 'gill-spacing'],inplace = True)

In [399]:
df_copy_1.isnull().sum()

id                          0
class                       0
cap-diameter                0
cap-shape                   0
cap-surface                 0
cap-color                   0
does-bruise-or-bleed        0
gill-attachment             0
gill-spacing                0
gill-color                  0
stem-height                 0
stem-width                  0
stem-root               18106
stem-surface            10618
stem-color                  0
veil-type               19180
veil-color              18343
has-ring                    0
ring-type                 759
spore-print-color       18240
habitat                     0
season                      0
dtype: int64

In [72]:
df_test = pd.read_csv('test.csv')

In [10]:
df_test

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,NaN,n,t,NaN,NaN,w,11.13,...,b,NaN,w,u,w,t,g,NaN,d,a
1,3116946,6.90,o,t,o,f,NaN,c,y,1.27,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,a
2,3116947,2.00,b,g,n,f,NaN,c,n,6.18,...,NaN,NaN,n,NaN,NaN,f,f,NaN,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,NaN,NaN,w,NaN,n,t,z,NaN,d,u
4,3116949,6.17,x,h,y,f,p,NaN,y,6.73,...,NaN,NaN,y,NaN,y,t,NaN,NaN,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,5194904,0.88,x,g,w,f,a,d,w,2.67,...,NaN,NaN,e,NaN,NaN,f,f,NaN,d,u
2077960,5194905,3.12,x,s,w,f,d,c,w,2.69,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
2077961,5194906,5.73,x,e,e,f,a,NaN,w,6.16,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
2077962,5194907,5.03,b,g,n,f,a,d,g,6.00,...,NaN,s,g,NaN,NaN,f,f,NaN,d,a


In [12]:
df_test.isnull().sum()

id                            0
cap-diameter                  7
cap-shape                    31
cap-surface              446904
cap-color                    13
does-bruise-or-bleed         10
gill-attachment          349821
gill-spacing             839595
gill-color                   49
stem-height                   1
stem-width                    0
stem-root               1838012
stem-surface            1321488
stem-color                   21
veil-type               1971545
veil-color              1826124
has-ring                     19
ring-type                 86195
spore-print-color       1899617
habitat                      25
season                        0
dtype: int64

In [73]:
columns = []
for col in df_test:
    if df_test[col].isnull().sum() <100 and col != 'id':
        columns.append(col)

In [34]:
columns

['cap-diameter',
 'cap-shape',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-color',
 'stem-height',
 'stem-width',
 'stem-color',
 'has-ring',
 'habitat',
 'season']

In [74]:
df_copy = pd.DataFrame()

In [78]:
for i in columns:
    if i in df.columns and df[i].dtypes == 'float64':
        
        df_copy[i]=df[i].fillna(df[i].mean())
    if i in df.columns:
        df_copy[i]=df[i]

In [79]:
df_copy.head()

,cap-diameter,cap-shape,cap-color,does-bruise-or-bleed,gill-color,stem-height,stem-width,stem-color,has-ring,habitat,season
0,8.80,f,u,f,w,4.51,15.39,w,f,d,a
1,4.51,x,o,f,n,4.79,6.48,o,t,d,w
2,6.94,f,b,f,w,6.85,9.93,n,f,l,w
3,3.88,f,g,f,g,4.16,6.53,w,f,d,u
4,5.85,x,w,f,w,3.37,8.36,w,f,g,a


In [80]:
df_copy.isnull().sum()

cap-diameter             4
cap-shape               40
cap-color               12
does-bruise-or-bleed     8
gill-color              57
stem-height              0
stem-width               0
stem-color              38
has-ring                24
habitat                 45
season                   0
dtype: int64

In [43]:
df_copy.dtypes

cap-diameter            float64
cap-shape                object
cap-color                object
does-bruise-or-bleed     object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
habitat                  object
season                   object
dtype: object

In [42]:
b = df_copy.fillna(df_copy.mean())

TypeError: can only concatenate str (not "int") to str

In [58]:
b = df_copy.apply(lambda df_copy.fillna(df_copy.mean()):df_copy.fillna(df_copy.mean()) if df_copy.dtypes == 'float64' )

SyntaxError: invalid syntax (31633876.py, line 1)

In [64]:
df_copy.dtypes("float64")

TypeError: 'Series' object is not callable

In [63]:
if all(df_copy.select_dtypes('float')): df_copy.fillna(df_copy.mean())

TypeError: can only concatenate str (not "int") to str

In [66]:
df_copy.select_dtypes('float').fillna(df_copy.mean())

TypeError: can only concatenate str (not "int") to str

In [86]:
df_copy['cap-diameter'].fillna(df_copy['cap-diameter'].mean(), inplace = True)

In [87]:
df_copy.isnull().sum()

cap-diameter             0
cap-shape               40
cap-color               12
does-bruise-or-bleed     8
gill-color              57
stem-height              0
stem-width               0
stem-color              38
has-ring                24
habitat                 45
season                   0
dtype: int64